## הפורמט המפורט ביותר (רשומה עבור כל מקטע דיבור בפרוטוקול)

```
עבור כל מקטע דיבור בפרוטוקול:
מס כנסת, תאריך, ועדה, הצח פרטית או ממשלתית (אם קיים), שם החוק (אם קיים), שם הדובר, שיוך הדובר, כמה מילים דיבר במקטע
שיוך הדובר –
שלב ראשון (הרלוונטי אליי)
 יו"ר הועדה, חברי כנסת, יועץ משפטי לוועדה, משרד המשפטים, משרד ממשלתי אחר, אחר
שלב שני (לבקשת איתי)
שיוך ח"כ לסיעה, אופוזיציה קואליציה, ארגון חברתי
```

## הפורמט המסוכם (רשומה עבור כל דובר בפרוטוקול )

```
עבור כל דובר בפרוטוקול:
מס כנסת, תאריך, ועדה, הצח פרטית או ממשלתית (אם קיים), שם החוק (אם קיים), שם הדובר, שיוך הדובר, מספר הפעמים שקיבל זכות דיבור, אחוז מספר המילים שאמר מסך כל המילים שנאמרו על ידי סך כל הדוברים בפרוטוקול, אורך זכות הדיבור הארוכה ביותר שלו.
```

## הערות

```
 לעניין היו"ר יופיע  בגוף הפרוטוקול "היו"ר שם פרטי שם משפחה
```

```
 לעניין חברי כנסת בחלק מהפרוטוקולים החדשים יופיע אחרי שם חבר הכנסת בסוגרים השייכות המפלגתית. בפרוטוקולים ישנים חברי הכנסת היו מופיעם במוזמנים אבל החל משנות 2000 ראיתי שהם מופעים תחת הכותרת חברי ועדה או חברי כנסת
```

```
 הגדרת היועץ המשפטי לוועדה משתנה בין פרוטוקולים. זה יכול להיות "יועץ משפטי" או "יועצת משפטית" או "ייעוץ משפטי" או "עוזרת ליועצת המשפטית" או "יועצות משפטיןת"
```

```
 בדרך כלל מופיע שם המוזמן – (מקף) תפקיד , משרד ממשלתי או בסדר הפוך משרד ממשלתי ואז תפקיד. לעיתים לפני שם המוזמן יכול להופיע עו"ד או תואר אחר והתואר לא יופיע אחר כך שהדובר מצוטט. לדוגמא "עו"ד מני מזוז – משרד המשפטים" ובטקסט בגוף הפרוטוקול יופע "מני מזוז:"  או " עו"ד סגן-ניצב  נורית זיו" ופרוטוקול "נורית זיו:"
```

## Install latest dataflows framework

In [7]:
%%bash
python3 -m pip install dataflows==0.0.64

You are using pip version 18.0, however version 19.3.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


Restart the kernel

## Generate the detailed speaker stats package

### Download the source package

In [6]:
%%bash
mkdir -p data/people_committees_meeting_speaker_stats
for FILENAME in datapackage.json kns_committeesession.csv speaker_stats.csv; do
    if [ -e data/people_committees_meeting_speaker_stats/${FILENAME} ]; then
        echo already exists:
            ls -lah data/people_committees_meeting_speaker_stats/${FILENAME}
    else
        curl -o data/people_committees_meeting_speaker_stats/${FILENAME} https://production.oknesset.org/pipelines/data/people/committees/meeting-speaker-stats/${FILENAME}
    fi
done

already exists:
-rw-r--r--    1 1001     root        5.6K Nov 30 08:12 data/people_committees_meeting_speaker_stats/datapackage.json
already exists:
-rw-r--r--    1 1001     root      117.3M Nov 30 08:13 data/people_committees_meeting_speaker_stats/kns_committeesession.csv
already exists:
-rw-r--r--    1 1001     root      483.1M Nov 30 08:17 data/people_committees_meeting_speaker_stats/speaker_stats.csv


## Load sessions data to memory

In [17]:
from dataflows import Flow, load, printer


SESSIONS = {}


def _load_sessions(row):
    SESSIONS[row['CommitteeSessionID']] = row


Flow(
    load('data/people_committees_meeting_speaker_stats/datapackage.json', resources='kns_committeesession'),
    _load_sessions,
    printer(tablefmt='html', num_rows=1)
).process()[1]

print('len(SESSIONS) = {}'.format(len(SESSIONS)))

#,CommitteeSessionID (integer),Number (integer),KnessetNum (integer),TypeID (integer),TypeDesc (string),CommitteeID (integer),Location (string),SessionUrl (string),BroadcastUrl (string),StartDate (datetime),FinishDate (datetime),Note (string),LastUpdatedDate (datetime),download_crc32c (string),download_filename (string),download_filesize (integer),parts_crc32c (string),parts_filesize (integer),parts_parsed_filename (string),text_crc32c (string),text_filesize (integer),text_parsed_filename (string),topics (array),committee_name (string),mks (array),invitees (array),legal_advisors (array),manager (array),financial_advisors (array),attended_mk_individual_ids (array)
1,64990,None,15,161,פתוחה,25,"חדר הוועדה, באגף קדמה, קומה 1, חדר 1720",http://main.knesset.gov.il/Activity/committees/Pages/AllCommitteesAgenda.aspx?Tab=3&ItemID=64990,None,2002-06-12 09:00:00,None,None,2011-04-12 05:28:59,None,None,None,None,None,None,None,None,None,"['חוק הבחירות לכנסת (תיקון מס\' 52), התשס""ד-2004']","ועדת החוקה, חוק ומשפט",None,None,None,None,None,[]
2,470756,None,18,161,פתוחה,661,"חדר הוועדה, באגף הוועדות (קדמה), קומה 2, חדר 2740",http://main.knesset.gov.il/Activity/committees/Pages/AllCommitteesAgenda.aspx?Tab=3&ItemID=470756,None,2012-06-06 09:30:00,None,None,2012-09-19 15:27:32,None,None,None,None,None,None,None,None,None,"['דו""ח מבקר המדינה על ההתמודדות עם המשט הטורקי ']",לענייני ביקורת המדינה,None,None,None,None,None,[]
...,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
74441,2051328,326,12,161,פתוחה,366,"חדר הוועדה, באגף קדמה, קומה 1, חדר 1720",http://main.knesset.gov.il/Activity/committees/Pages/AllCommitteesAgenda.aspx?Tab=3&ItemID=2051328,None,1991-12-30 00:00:00,1991-12-30 00:00:00,"בקשת חה""כ עוזי לנדאו לרביזיה בהחלטת הוועדה לדחות בקשתם של תשעה חברי כנסת להוסיף הסתייגות לסעיף 56 בה ...",2019-11-26 16:43:12,GvGY9w==,files/23/4/1/415524.DOC,188416,NSe4Yw==,82566,files/2/0/2051328.csv,Rv/MjQ==,82603,files/2/0/2051328.txt,['חוק-יסוד: הממשלה'],"ועדת החוקה, חוק ומשפט",[],[],[],[],[],[]


len(SESSIONS) = 74441


In [32]:
from dataflows import Flow, load, printer, update_resource

num_session_items = 0
last_session = None

for row in Flow(
    load('https://production.oknesset.org/pipelines/data/committees/kns_cmtsessionitem/datapackage.json')
).results()[0][0]:
    session = SESSIONS.get(row['CommitteeSessionID'])
    if session:
        session.setdefault('items', []).append(row)
        num_session_items += 1
        last_session_item = row

Flow(
    iter([last_session_item]), update_resource('res_1', name='session.items'), printer(tablefmt='html')
).process()
        
print('num_session_items = {}'.format(num_session_items))

#,CmtSessionItemID (integer),ItemID (integer),CommitteeSessionID (integer),Ordinal (integer),StatusID (integer),Name (string),ItemTypeID (integer),LastUpdatedDate (datetime)
1,869954,2078438,2044137,1,113,"חוק התקציב לשנת הכספים 1983 (מס' 2), התשמ""ד-1984",2,2019-12-02 12:03:22


num_session_items = 47705


## Relate sessions to bills/laws

In [33]:
from collections import defaultdict
from dataflows import Flow, load, printer

ITEM_TYPES = {}

for row in Flow(
    load('https://production.oknesset.org/pipelines/data/knesset/kns_itemtype/datapackage.json')
).results()[0][0]:
    ITEM_TYPES[row['ItemTypeID']] = row
    
print('len(ITEM_TYPES) = {}'.format(len(ITEM_TYPES)))

len(ITEM_TYPES) = 10


In [34]:
from dataflows import Flow, load, printer, update_resource

Flow(iter(ITEM_TYPES.values()), update_resource('res_1', name='ITEM_TYPES'), printer(tablefmt='html')).process()[1]

#,ItemTypeID (integer),Desc (string),TableName (string)
1,1,שאילתה,KNS_Query
2,2,הצעת חוק,KNS_Bill
3,3,הצעת אי אמון,None
4,4,הצעה לסדר היום,KNS_Agenda
5,5,ישיבת מליאה,KNS_PlenumSession
6,12,ישיבת ועדה,KNS_CommitteSession
7,6000,פעולה על פי חוק,KNS_Law
8,6001,תיקון טעות,KNS_Law
9,6002,חוק בן חיצוני,KNS_Law
10,6003,חוק אב,KNS_IsraelLaw


{}

## Load bill/lawdata

In [35]:
kns_bill_rows, kns_law_binding_rows, kns_law_rows, kns_israel_law_rows = Flow(
    load('https://production.oknesset.org/pipelines/data/bills/kns_bill/datapackage.json'),
    load('https://production.oknesset.org/pipelines/data/laws/kns_law_binding/datapackage.json'),
    load('https://production.oknesset.org/pipelines/data/laws/kns_law/datapackage.json'),
    load('https://production.oknesset.org/pipelines/data/laws/kns_israel_law/datapackage.json'),
).results()[0]

In [36]:
bills = {}
for bill in kns_bill_rows:
    bills[bill['BillID']] = bill

In [37]:
laws = {}
for law in kns_law_rows:
    laws[law['LawID']] = law

In [38]:
israel_laws = {}
for israel_law in kns_israel_law_rows:
    israel_laws[israel_law['IsraelLawID']] = israel_law

In [39]:
for law_binding in kns_law_binding_rows:
    law = laws.get(law_binding['LawID'])
    if law:
        law.setdefault('law_bindings', []).append(law_binding)

In [40]:
kns_israel_law_binding_rows = Flow(load('https://production.oknesset.org/pipelines/data/laws/kns_israel_law_binding/datapackage.json')).results()[0][0]

In [41]:
match = 0
no_match = 0
for israel_law_binding in kns_israel_law_binding_rows:
    israel_law = israel_laws.get(israel_law_binding['IsraelLawID'])
    if israel_law:
        match += 1
        israel_law.setdefault('israel_law_bindings', []).append(israel_law_binding)
    else:
        no_match +=1
print('match: {}, no match: {}'.format(match, no_match))        

match: 353, no match: 0


In [45]:
from dataflows import Flow, load, printer, update_resource

for title, rows in {
    'bills': bills.values(), 
    'laws': laws.values(), 
    'israel_laws': israel_laws.values(),
}.items():
    Flow(
        iter(rows), update_resource('res_1', name=title), 
        printer(tablefmt='html', num_rows=1)
    ).process()[1]

#,BillID (integer),KnessetNum (integer),Name (string),SubTypeID (integer),SubTypeDesc (string),PrivateNumber (integer),CommitteeID (integer),StatusID (integer),Number (integer),PostponementReasonID (integer),PostponementReasonDesc (string),PublicationDate (any),MagazineNumber (any),PageNumber (any),IsContinuationBill (any),SummaryLaw (any),PublicationSeriesID (any),PublicationSeriesDesc (any),PublicationSeriesFirstCall (string),LastUpdatedDate (datetime)
1,426521,18,"הצעת חוק התכנון והבניה (תיקון מס' 100) (חובת התקנת מערכת סולרית), התשע""ב-2012",54,פרטית,3840,656,113,471,None,None,None,None,None,None,None,None,None,"הצ""ח הכנסת (מתשס""ג 10/2002) - 471 ,מיום 11:00:00",2013-05-23 18:37:56
2,426868,18,"הצעת חוק העזרה הרפואית הבסיסית, התשע""ב-2012",54,פרטית,4046,None,104,None,None,None,None,None,None,None,None,None,None,None,2013-05-23 18:37:56
...,,,,,,,,,,,,,,,,,,,,
44393,2083334,22,"הצעת חוק יסודות התקציב (תיקון - תקציב הכנסת) (הוראת שעה), התש""ף-2019",54,פרטית,361,2111,108,None,None,None,None,None,None,None,None,None,None,None,2019-12-02 14:54:24


#,LawID (integer),TypeID (integer),TypeDesc (string),SubTypeID (integer),SubTypeDesc (string),KnessetNum (integer),Name (string),PublicationDate (datetime),PublicationSeriesID (integer),PublicationSeriesDesc (string),MagazineNumber (string),PageNumber (string),LastUpdatedDate (datetime),law_bindings (array)
1,2001421,6001,תיקון טעות,6051,"תיקון לפי ס' 10א לפשס""מ – אחרי פרסום החוק",10,"תיקון טעות בחוק הבזק, התשמ""ב-1982",1984-01-04 00:00:00,6071,ספר החוקים,1102,1102,2014-09-10 14:27:04,"[{'LawBindingID': 41559, 'LawID': 2001421, 'LawTypeID': 6001, 'IsraelLawID': 2000002, 'ParentLawID': ..."
2,2001422,6001,תיקון טעות,6051,"תיקון לפי ס' 10א לפשס""מ – אחרי פרסום החוק",12,"תיקון טעות בחוק הבזק (תיקון מס' 6), התשמ""ב-1982",1989-12-07 00:00:00,6071,ספר החוקים,1291,1291,2014-09-10 14:27:04,"[{'LawBindingID': 41565, 'LawID': 2001422, 'LawTypeID': 6001, 'IsraelLawID': 2000002, 'ParentLawID': ..."
...,,,,,,,,,,,,,,
1612,2001734,6002,חוק בן חיצוני,6043,מנדטורי,None,"פקודת בריאות העם (תקנות בנידון צרכי מאכל), 1945",1945-09-04 00:00:00,6078,עיתון רשמי מנדטורי,תוס' 1 - 1436,תוס' 1 - 1436,2019-11-27 11:56:34,"[{'LawBindingID': 47461, 'LawID': 2001734, 'LawTypeID': 6002, 'IsraelLawID': 2000838, 'ParentLawID': ..."


#,IsraelLawID (integer),KnessetNum (integer),Name (string),IsBasicLaw (integer),IsFavoriteLaw (integer),IsBudgetLaw (any),PublicationDate (datetime),LatestPublicationDate (datetime),LawValidityID (integer),LawValidityDesc (string),ValidityStartDate (any),ValidityStartDateNotes (any),ValidityFinishDate (any),ValidityFinishDateNotes (any),LastUpdatedDate (datetime)
1,2000001,1,"חוק רשות הפיתוח (העברת נכסים), התש""י-1950",False,False,None,1950-08-09 00:00:00,1960-07-29 00:00:00,6079,תקף,None,None,None,None,2014-09-10 14:26:43
2,2000005,None,"דבר המלך על הטיס במושבות (הטלת חוקים), 1937",False,False,None,1937-07-29 00:00:00,2011-04-13 00:00:00,6079,תקף,None,None,None,None,2014-09-10 14:26:43
...,,,,,,,,,,,,,,,
1730,2002355,20,"חוק אומנה לילדים, התשע""ו-2016",False,False,False,2016-03-06 00:00:00,2017-01-10 00:00:00,6079,תקף,2016-12-06 00:00:00,None,None,None,2019-11-25 15:10:16


## Identify session bill/law type

In [47]:
matched_bills = 0
mismatch_laws = 0
mismatch_israel_laws = 0
for session in SESSIONS.values():
    session['bill_types'] = set()
    session['bill_names'] = set()
    session['law_names'] = set()
    for item in session.get('items', []):
        if item['ItemTypeID'] == 2:
            bill = bills[item['ItemID']]
            session['bill_types'].add(bill['SubTypeDesc'])
            session['bill_names'].add(bill['Name'])
            matched_bills += 1
        elif item['ItemTypeID'] in [6000,6001,6002]:
            law = laws.get(int(item['ItemID']))
            if law:
                raise Exception(law)
            else:
                mismatch_laws += 1
        elif item['ItemTypeID'] == 6003:
            israel_law = israel_laws.get(int(item['ItemID']))
            if israel_law:
                raise Exception(israel_law)
            else:
                mismatch_israel_laws += 1
print('mismatch laws: {}, mismatch israel laws: {}'.format(mismatch_laws, mismatch_israel_laws))
print('matched_bills: {}'.format(matched_bills))

mismatch laws: 4650, mismatch israel laws: 0
matched_bills: 44679


## Generate the detailed package

output data is spread over the following directory structure:
    
```
./data/detailed_speaker_parts/Knesset{KnessetNum}/Committee{committee_name}/datapackage.json
```

## (optional) Clear existing data / checkpoints

In [109]:
%%bash
rm -rf .checkpoints/detailed_speaker_parts_input_sorted
rm -rf data/detailed_speaker_parts

In [ ]:
from dataflows import Flow, load, printer, add_field, dump_to_path, sort_rows, update_resource, checkpoint
from collections import defaultdict


LIMIT_ROWS=None

# מס כנסת, תאריך, ועדה, הצח פרטית או ממשלתית (אם קיים), שם החוק (אם קיים), שם הדובר, שיוך הדובר, מספר הפעמים שקיבל זכות דיבור,
# אחוז מספר המילים שאמר מסך כל המילים שנאמרו על ידי סך כל הדוברים בפרוטוקול, אורך זכות הדיבור הארוכה ביותר שלו# # .


def _dump_sessions_parts_to_path(parts):
    first_session = SESSIONS[parts[0]['CommitteeSessionID']]
    target_path = './data/detailed_speaker_parts/Knesset{}/{}/'.format(first_session['KnessetNum'], first_session['committee_name'][:60])
    parts_session_ids = set()
    for part in parts:
        session = SESSIONS[part['CommitteeSessionID']]
        if part['CommitteeSessionID'] not in parts_session_ids:
            parts_session_ids.add(part['CommitteeSessionID'])
            session['headers'] = {}
            session['headers_stats'] = {
                'num_words': 0,
                'num_parts': 0
            }
        header = part['header']
        if header is not None and len(header.strip()) > 2:
            if not session['headers'].get(header):
                session['headers'][header] = {
                    'header': header, 
                    'categories': set(),
                    'num_words': 0,
                    'num_parts': 0,
                    'longest_part_num_words': 0
                }
            header = session['headers'][header]
            header['categories'].add(part['part_category'])
            header['num_parts'] += 1
            session['headers_stats']['num_parts'] += 1
            header['num_words'] += part['body_num_words']
            session['headers_stats']['num_words'] += part['body_num_words']
            if part['body_num_words'] > header['longest_part_num_words']:
                header['longest_part_num_words'] = part['body_num_words']
    num_headers = 0
    for session_id in parts_session_ids:
        session = SESSIONS[session_id]
        for header in session['headers'].values():
            num_headers += 1
            header['categories'] = [cat for cat in header['categories'] if cat]
            header['session_num_words'] = session['headers_stats']['num_words']
            header['session_num_parts'] = session['headers_stats']['num_parts']
            header['percent_num_words'] = round(header['num_words'] / header['session_num_words'] * 100, 2)
    print('Saving {} parts, {} headers to {}'.format(len(parts), num_headers, target_path))
    
    def _header_iterator():        
        for session_id in parts_session_ids:
            session = SESSIONS[session_id]
            if len(session['headers']) > 0:
                for header in session['headers'].values():
                    if (
                        (
                            len(header['categories']) == 0 
                            and header['num_parts'] == 1 
                            and header['percent_num_words'] < 0.5
                        ) or header['header'].strip() in [
                            'מוזמנים',
                            'סדר היום',
                            'מנהלת הועדה',
                            'מנהלת הוועדה'
                            
                        ]
                    ):
                        pass
                    else:
                        yield {
                            'KnessetNum': session.get('KnessetNum', -1),
                            'StartDate': session.get('StartDate', None),
                            'committee_name': session.get('committee_name', ''),
                            'bill_types': list(session.get('bill_types', [])),
                            'bill_names': list(session.get('bill_names', [])),
                            **header
                        }

    Flow(
        iter(parts),
        _header_iterator(),
        update_resource('res_1', name='parts', path='parts.csv'),
        update_resource('res_2', name='headers', path='headers.csv'),
        dump_to_path(target_path)
    ).process()    
    for session_id in parts_session_ids:
        SESSIONS[session_id]['headers'] = None
        SESSIONS[session_id]['headers_stats'] = None


def _dump_to_paths(rows):
    current_KnessetNumCommitteeName = None
    current_parts = []
    for row in rows:
        session = SESSIONS.get(row['CommitteeSessionID'])
        if session:
            yield row
            if '{}{}'.format(session['KnessetNum'],session['committee_name']) != current_KnessetNumCommitteeName:
                if current_KnessetNumCommitteeName is not None:
                    assert len(current_parts) > 0
                    _dump_sessions_parts_to_path(current_parts)
                current_KnessetNumCommitteeName = '{}{}'.format(session['KnessetNum'],session['committee_name'])
                current_parts = [row]
            else:
                current_parts.append(row)
    if len(current_parts) > 0:
        _dump_sessions_parts_to_path(current_parts)


Flow(
    load('data/people_committees_meeting_speaker_stats/datapackage.json', resources='speaker_stats', limit_rows=LIMIT_ROWS),
    *[
        add_field(name=n, type=t) for n, t in {
            'KnessetNum': 'number',
            'committee_name': 'string',
            'StartDate': 'datetime',
            'bill_types': 'array',
            'bill_names': 'array'
        }.items()
    ],
    lambda row: row.update(KnessetNum=SESSIONS.get(row['CommitteeSessionID'], {}).get('KnessetNum', -1),
                           committee_name=SESSIONS.get(row['CommitteeSessionID'], {}).get('committee_name', ''),
                           StartDate=SESSIONS.get(row['CommitteeSessionID'], {}).get('StartDate', None),
                           bill_types=list(SESSIONS.get(row['CommitteeSessionID'], {}).get('bill_types', [])),
                           bill_names=list(SESSIONS.get(row['CommitteeSessionID'], {}).get('bill_names', []))),
    sort_rows('{KnessetNum}{committee_name}{StartDate}{part_index}'),
    checkpoint('detailed_speaker_parts_input_sorted'),
    _dump_to_paths,
    printer(tablefmt='html', num_rows=1)
).process()[1]

saving checkpoint to: .checkpoints/detailed_speaker_parts_input_sorted


Saving 15359 parts, 8011 headers to ./data/detailed_speaker_parts/Knesset11/החינוך והתרבות/
Saving 10663 parts, 4300 headers to ./data/detailed_speaker_parts/Knesset11/הכלכלה/
Saving 14240 parts, 6968 headers to ./data/detailed_speaker_parts/Knesset11/הכנסת/
Saving 74635 parts, 28489 headers to ./data/detailed_speaker_parts/Knesset11/הכספים/
Saving 32150 parts, 13901 headers to ./data/detailed_speaker_parts/Knesset11/העבודה והרווחה/
Saving 2957 parts, 1736 headers to ./data/detailed_speaker_parts/Knesset11/העליה והקליטה/
Saving 15929 parts, 7813 headers to ./data/detailed_speaker_parts/Knesset11/הפנים ואיכות הסביבה/
Saving 13197 parts, 5698 headers to ./data/detailed_speaker_parts/Knesset11/ועדת החוקה, חוק ומשפט/
Saving 17541 parts, 7178 headers to ./data/detailed_speaker_parts/Knesset11/לענייני ביקורת המדינה/
Saving 22340 parts, 8891 headers to ./data/detailed_speaker_parts/Knesset12/החינוך והתרבות/
Saving 30752 parts, 10841 headers to ./data/detailed_speaker_parts/Knesset12/הכלכלה/
S

## Create zip archive

In [114]:
%%bash
apk add zip
zip -r ./data/detailed_speaker_parts.zip ./data/detailed_speaker_parts

fetch http://dl-cdn.alpinelinux.org/alpine/v3.8/main/x86_64/APKINDEX.tar.gz
fetch http://dl-cdn.alpinelinux.org/alpine/v3.8/community/x86_64/APKINDEX.tar.gz
(1/2) Installing unzip (6.0-r4)
(2/2) Installing zip (3.0-r6)
Executing busybox-1.28.4-r0.trigger
OK: 345 MiB in 90 packages
  adding: data/detailed_speaker_parts/ (stored 0%)
  adding: data/detailed_speaker_parts/Knesset13/ (stored 0%)
  adding: data/detailed_speaker_parts/Knesset13/הפנים ואיכות הסביבה/ (stored 0%)
  adding: data/detailed_speaker_parts/Knesset13/הפנים ואיכות הסביבה/parts.csv (deflated 90%)
  adding: data/detailed_speaker_parts/Knesset13/הפנים ואיכות הסביבה/datapackage.json (deflated 86%)
  adding: data/detailed_speaker_parts/Knesset13/הפנים ואיכות הסביבה/headers.csv (deflated 85%)
  adding: data/detailed_speaker_parts/Knesset13/ועדת החוקה, חוק ומשפט/ (stored 0%)
  adding: data/detailed_speaker_parts/Knesset13/ועדת החוקה, חוק ומשפט/parts.csv (deflated 91%)
  adding: data/detailed_speaker_parts/Knesset13/ועדת החוקה,